In [1]:
!pip install opencv-python

In [45]:
import tensorflow as tf
import json
import pandas as pd
import numpy as np
from skimage import io, transform
from sklearn.model_selection import train_test_split
import cv2
from tensorflow.keras import datasets, layers, models, losses, Model
from tqdm.notebook import tqdm
import os
import glob

In [3]:
#from google.colab import drive
#drive.mount('/content/drive')

In [4]:
#!cp "drive/MyDrive/all.json" "/content/"

In [8]:
with open("all.json", 'r') as f:
    data_df = pd.json_normalize([i for i in json.loads(f.read()) if i is not None])

In [9]:
def reshape(img):
    arr = io.imread(url)
    if arr.shape!=(128,256,3):
        arr = (255 * transform.resize(arr, (128,256))).astype(np.uint8)
    return arr

In [28]:
# images = {}
# folder = "img2"
# for filename in tqdm(os.listdir(folder)):
#     img = cv2.imread(os.path.join(folder,filename))
#     if img is not None:
#         if img.shape!=(128,256,3):
#             img = (255 * transform.resize(img, (128,256))).astype(np.uint8)
#         images[os.path.splitext(filename)[0]] = img
        

def reshape(img):
    if img.shape!=(128,256,3):
        img = (255 * transform.resize(img, (128,256))).astype(np.uint8)
    return img

In [46]:
images = {file: cv2.imread(file) for file in tqdm(glob.glob("img2/*.jpg"))}

  0%|          | 0/76440 [00:00<?, ?it/s]

In [50]:
images.keys()

dict_keys(['img2\\1000148914149750.jpg', 'img2\\1000165370829738.jpg', 'img2\\1000308077173457.jpg', 'img2\\1000346054206532.jpg', 'img2\\1000390084038009.jpg', 'img2\\1000458640591153.jpg', 'img2\\1000476557324429.jpg', 'img2\\1000570843813146.jpg', 'img2\\1000755687567152.jpg', 'img2\\1000887294078575.jpg', 'img2\\1000903354050600.jpg', 'img2\\1000917240613461.jpg', 'img2\\1000959820713563.jpg', 'img2\\1001086410698412.jpg', 'img2\\1001150390748438.jpg', 'img2\\1001288454596480.jpg', 'img2\\1001443240391160.jpg', 'img2\\1001533467274221.jpg', 'img2\\1001562847255564.jpg', 'img2\\1001608897386849.jpg', 'img2\\1001641880373446.jpg', 'img2\\1001657243973450.jpg', 'img2\\1001743017472372.jpg', 'img2\\1001943897339542.jpg', 'img2\\1002095757430645.jpg', 'img2\\1002183200601634.jpg', 'img2\\1002234643645314.jpg', 'img2\\1002274530582161.jpg', 'img2\\1002375903838500.jpg', 'img2\\1002556066947060.jpg', 'img2\\1002658580473274.jpg', 'img2\\1002803506792284.jpg', 'img2\\1002887367272397.jpg',

In [34]:
os.path.splitext(os.path.basename('img2\\1000148914149750.jpg'))[0]

'1000148914149750'

In [47]:
sss=pd.Series({"a":1,"b":2,"c":3})
sss.index

Index(['a', 'b', 'c'], dtype='object')

In [35]:
data_df

,id,url_256,url_1024,coordinates
0,1509662729539070,https://scontent.fsin14-2.fna.fbcdn.net/m1/v/t...,https://scontent.fsin14-2.fna.fbcdn.net/m1/v/t...,"[0, 0]"
1,1587956288269248,https://scontent.fsin14-2.fna.fbcdn.net/m1/v/t...,https://scontent.fsin14-2.fna.fbcdn.net/m1/v/t...,"[0, 0]"
2,1414341782427856,https://scontent.fsin14-2.fna.fbcdn.net/m1/v/t...,https://scontent.fsin14-2.fna.fbcdn.net/m1/v/t...,"[0, 0]"
3,1504003806771142,https://scontent.fsin14-2.fna.fbcdn.net/m1/v/t...,https://scontent.fsin14-2.fna.fbcdn.net/m1/v/t...,"[-9.5395717634566e-07, 5.4042628244286e-08]"
4,1502346620201016,https://scontent.fsin14-2.fna.fbcdn.net/m1/v/t...,https://scontent.fsin14-2.fna.fbcdn.net/m1/v/t...,"[0, 0]"
...,...,...,...,...
79074,4204140206347239,https://scontent.fsin14-2.fna.fbcdn.net/m1/v/t...,https://scontent.fsin14-2.fna.fbcdn.net/m1/v/t...,"[99.206494380421, 9.4558343148341]"
79075,1405412416480572,https://scontent.fsin14-2.fna.fbcdn.net/m1/v/t...,https://scontent.fsin14-2.fna.fbcdn.net/m1/v/t...,"[99.169389331325, 9.3891869120472]"
79076,3963740523712531,https://scontent.fsin14-2.fna.fbcdn.net/m1/v/t...,https://scontent.fsin14-2.fna.fbcdn.net/m1/v/t...,"[99.114069704404, 9.6701075610815]"
79077,1375948262803916,https://scontent.fsin14-2.fna.fbcdn.net/m1/v/t...,https://scontent.fsin14-2.fna.fbcdn.net/m1/v/t...,"[99.128083068806, 9.5891300264083]"


In [ ]:
coords = np.stack(data_df["coordinates"][:100000])

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(images, coords, test_size=0.20)

In [ ]:
base_model = tf.keras.applications.ResNet152(weights = 'imagenet', include_top = False, input_shape = (128,256,3))
for layer in base_model.layers:
  layer.trainable = False
x = layers.Flatten()(base_model.output)
x = layers.Dense(1000, activation='relu')(x)
predictions = layers.Dense(2, activation = 'linear')(x)

In [ ]:
head_model = Model(inputs = base_model.input, outputs = predictions)
head_model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])

In [ ]:
history = head_model.fit(X_train, y_train, batch_size=64, epochs=40)